# Exploration du jeu d'images

L'objectif de ce notebook est d'explorer le jeu d'images du usecase et d'identifier les différents biais. A l'issue de ce notebook, le dataset d'image aura été nettoyé.
les inputs de ce notebook sont :
- le jeu d'images obtenu suite à de l'etape de nettoyage des données .csv
- le fichier .csv correspondant au jeu d'images

In [ ]:
import os
import numpy as np
import PIL
from PIL import Image, ImageStat
import cv2
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import shutil
%matplotlib inline
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
# définition du dossier image source du usecase
full_images = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\dataset_cleaned'

# définition du chemin d'accés du dataframe contenant les infos de l'étude
dataframe_dataset = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\cleaned_dataset.csv'

In [ ]:
df = pd.read_csv(dataframe_dataset)
df_filtered = df['filename'].sample(frac = 1/50)
test_resolution = df_filtered.tolist()
print(len(test_resolution))

## Quelques informations générales sur le jeu de données d'images

In [ ]:
# récupération des informations sur la taille des images
sizes = []
for file in os.listdir(full_images):
    if os.path.isfile(os.path.join(full_images, file)) and file.endswith(".jpg"):
        sizes.append(os.path.getsize(os.path.join(full_images, file)))

In [ ]:
# récupération des informations sur le contrast et la luminosité des images
def get_image_contrast_brightness(image_path):
    # Load the image
    img = cv2.imread(image_path)                         # Convert the image to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)     # Calculate the standard deviation of the pixel intensities
    contrast = np.std(img_gray)                          # Calculate the average pixel brightness
    brightness = np.mean(img_gray)    
    return contrast, brightness

contrast_list = []
brightness_list = []
for file in os.listdir(full_images):
    if os.path.isfile(os.path.join(full_images, file)) and file.endswith(".jpg"):
        file_path = os.path.join(full_images, file)
        contrast,brightness = get_image_contrast_brightness(file_path)
        contrast_list.append(contrast)
        brightness_list.append(brightness)

In [ ]:
# identification du format de couleur des images
def is_grayscale(image_path):
    img = Image.open(image_path).convert('RGB')
    w, h = img.size
    for i in range(w):
        for j in range(h):
            r, g, b = img.getpixel((i,j))
            if r != g != b:
                return False
    return True

greyscale_list = []
rgb_list = []
for file in os.listdir(full_images):
    if os.path.isfile(os.path.join(full_images, file)) and file.endswith(".jpg"):
            if (is_grayscale(os.path.join(full_images, file))):
                greyscale_list.append(file)
            else:
                rgb_list.append(file)  


In [ ]:
print(len(sizes), len(contrast_list), len(brightness_list), len(greyscale_list), len(rgb_list))

#### création d'une figure résumant ces informations

In [ ]:
sns.set()
plt.rcParams['figure.figsize'] = [8, 8]

# Créer la figure
fig = plt.figure()

plt.gcf().subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, wspace = 0.3, hspace = 0.3)
# Créer les 4 graphiques
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)
ax3 = fig.add_subplot(2, 2, 3)
ax4 = fig.add_subplot(2, 2, 4)

# Tracer les données sur les graphiques
ax1.hist(x = sizes, bins = 100, density = True)
ax2.bar(list_labels, list_sizes)
ax3.hist(x = contrast_list, bins = 100, density = True)
ax4.hist(x = brightness_list, bins = 100, density = True)

ax1.set_xlabel('Taille des fichiers en octets')
ax1.set_ylabel('Densité de fichiers')
ax2.set_xlabel('Nuance de couleur')
ax2.set_ylabel('Nombre de fichiers')
ax2.set_yscale('log')
ax3.set_ylabel('Densité')
ax3.set_xlabel('distribution du contraste')
ax4.set_ylabel('Densité')
ax4.set_xlabel('distribution de la luminosité')

plt.show()


Les images en nuance de gris pourront etre supprimées car leur proportion est trés faible dans le dataset.
Les images sont de petites dimensions, la distribution du contraste et de la luminosité sont de type gaussien.

## Identification des images sur fond blanc 

L'objectif est ici de créer une liste des images sur fond blanc (images qui ne sont probablement pas des photos réelles) et qui risquent de fausser l'entrainement d'un modèle.

In [ ]:
# création d'une fonction permettant d'identifier les images ayant les pixels des angles blancs
def has_white_background(image: np.ndarray) -> bool:
    """
    Retourne True si les coin de l'imaage sont blancs, False sinon
    image: image as a NumPy array.
     """
    # récupération des dimensions de l'image
    height, width = image.shape

    # Contrôle des valeurs des pixels des angles
    if (np.all(image[0, 0] == 255) and
        np.all(image[0, width-1] == 255) and
        np.all(image[height-1, 0] == 255) and
        np.all(image[height-1, width-1] == 255)):
        return True
    else:
        return False

# création d'une liste d'images à fond blanc
white_background = []
for subdir, _, files in os.walk(full_images):
    for file in files:
        if file.endswith('.jpg'):
            with Image.open(os.path.join(subdir, file)) as img:
                if has_white_background(np.array(img.convert('L'))):
                    white_background.append(file)
print(len(white_background))

# création de fonctions permettant de générer une grille de 100 images avec des transformations (rotation) aléatoires
def get_side(img, side_type, n = 5):
    h, w, c = img.shape
    if side_type == "horizontal":
        return np.ones((h,n,c))
    return np.ones((n,w,c))

def show_gallery(im_ls,n=5, shuffle=True):
    images = []
    vertical_images = []
    if shuffle:
        random.shuffle(im_ls)
    vertical_images = []
    for i in range(n*n):
        img = load_img(os.path.join(IMAGE_DIR,im_ls[i]), target_size=(W,H))
        img = img_to_array(img)
        hside = get_side(img,side_type="horizontal")
        images.append(img)
        images.append(hside)
        
        if (i+1) % n == 0:
            himage=np.hstack((images))
            vside = get_side(himage, side_type="vertical")
            vertical_images.append(himage)
            vertical_images.append(vside)
            
            images = []
        
    gallery = np.vstack((vertical_images))
    plt.figure(figsize=(20,20))
    plt.axis("off")
    plt.imshow(gallery.astype(np.uint8))
    plt.show()
    
# observation d'une grille d'images sur fond blanc
IMAGE_DIR = full_images
W, H = 224, 224
show_gallery(white_background, n=10)


Les images de la liste 'white_background' pourront être supprimées car ce sont essentiellement des dessins et schémas qui risquent de fausser l'apprentissage.

In [ ]:
# création d'une liste d'images à supprimer
images_to_delete = white_background

## Identifcation des images sur fond noir

L'objectif est ici de créer une liste des images prises au microscope (fond noir + cercle de prise de vue) afin de pouvoir les écarter du jeu de données qui servira pour les entrainements de modèles

In [ ]:
# création d'une fonction permettant d'identifier les images ayant les pixels des angles noirs
def has_black_background(image: np.ndarray) -> bool:
    """
    Retourne True si le fond de l'image est noir, False sinon.
    image: image as a NumPy array.
    """
    # Recupération des dimensions de l'image
    height, width = image.shape

    # Contrôle des valeurs des pixels de angles
    if (np.all(image[0, 0] == 0) and
        np.all(image[0, width-1] == 0) and
        np.all(image[height-1, 0] == 0) and
        np.all(image[height-1, width-1] == 0)):
        return True
    else:
        return False

# création d'une liste des images ayant les pixels des angles moirs
black_background_images = []
for subdir, _, files in os.walk(full_images):
    for file in files:
        if file.endswith('.jpg'):
            with Image.open(os.path.join(subdir, file)) as img:
                if has_black_background(np.array(img.convert('L'))):
                    black_background_images.append(file)
print(len(black_background_images))

# affichage d'une grille d'images ayant un fond noir
show_gallery(black_background_images, n=4)

Les images de la liste 'black_background_images' pourront être supprimées car ce sont essentiellement d'images prises au microscope qui risquent de fausser l'apprentissage et la perte d'images non aquises au microscope reste trés faible.

In [ ]:
# mise à jour de la liste d'images à supprimer
for filename in black_background_images:
    images_to_delete.append(filename)


## Recherche de nettoyage des données sur le format des images

In [ ]:
# création d'un dataframe avec les résolutions des images
resolutions = []

for file in os.listdir(full_images):
    image = PIL.Image.open(os.path.join(full_images, file))
    W, H = image.width, image.height
    resolution = [W, H]
    resolutions.append(resolution)

In [ ]:
# tri des résolutions
def tri_resolutions(liste):
    '''
  Trie une liste de couples de chiffres en ordonnant pour chaque couple de chiffre la plus grande des deux valeurs en premier.
  Args:
    liste: La liste des résolutions à trier.
  Returns:
    La liste des résolutions triées.
'''
    for resolution in liste:
        if resolution[0] < resolution[1]:
            resolution[0], resolution[1] = resolution[1], resolution[0]
    return resolutions

tri_resolutions(resolutions)

# création d'un dataframe avec les résolutions des images
width = []
height = []
for resolution in resolutions:
    width.append(resolution[0])
    height.append(resolution[1])

files = os.listdir(full_images)

# construction du dataframe
data = {'filename' : pd.Series(files), 'width' : pd.Series(width), 'height' : pd.Series(height)}
df_resolutions = pd.DataFrame(data = data)
# ajout d'une colonne concatenant largeur et hauteur pour chaque image
df_resolutions['image_size'] = df_resolutions['width'].astype(str) + 'x' + df_resolutions['height'].astype(str)
# sauvegarde du dataframe
dossier_parent = os.path.dirname(full_images)
os.chdir(dossier_parent)
df_resolutions.to_csv(f"{os.getcwd()}/df_resolutions.csv")


# création d'un barplot des résolutions les plus représentées

# Obtenir les 10 catégories les plus représentées
categories = df_resolutions["image_size"].value_counts().head(10).index
# Créer une liste des noms des catégories
noms_categories = []
for categorie in categories:
    noms_categories.append(categorie)
noms_categories = list(reversed(noms_categories))
# Créer une liste des valeurs des catégories
valeurs_categories = []
for categorie in categories:
    valeurs_categories.append(df_resolutions["image_size"].value_counts()[categorie] / len(df_resolutions))
valeurs_categories = list(reversed(valeurs_categories))
# Obtenir les valeurs des autres catégories
autres_categories = df_resolutions["image_size"].value_counts()[categories[-1]:].sum() / len(df_resolutions)
# Créer une liste des valeurs des autres catégories
valeurs_autres_categories = [autres_categories]
# Créer un objet `Bar()`
bar = plt.barh(noms_categories + ["Autres"], valeurs_categories + valeurs_autres_categories)
# Définir les paramètres de l'objet `Bar()`
plt.title("Les 10 résolutions les plus représentées")
plt.xlabel("Résolutions")
plt.ylabel("Fréquences")
# Afficher l'objet `Bar()`
plt.show()

In [ ]:
df_resolutions.info()

Afin de limiter le nombre d'images pour un entrainement, seules les images des 5 résolutions les plus représentées pourraient être conservées...

### Recherche de spécificitées sur les 10 formats principaux

In [ ]:
# Creation des listes d'images de resolutions spécifiques
r_320x240 = df_resolutions.loc[df_resolutions['image_size'] == '320x240', 'filename'].tolist()
r_320x213 = df_resolutions.loc[df_resolutions['image_size'] == '320x213', 'filename'].tolist()
r_320x214 = df_resolutions.loc[df_resolutions['image_size'] == '320x214', 'filename'].tolist()
r_320x180 = df_resolutions.loc[df_resolutions['image_size'] == '320x180', 'filename'].tolist()
r_320x212 = df_resolutions.loc[df_resolutions['image_size'] == '320x212', 'filename'].tolist()
r_320x239 = df_resolutions.loc[df_resolutions['image_size'] == '320x239', 'filename'].tolist()
r_320x320 = df_resolutions.loc[df_resolutions['image_size'] == '320x320', 'filename'].tolist()
r_320x211 = df_resolutions.loc[df_resolutions['image_size'] == '320x211', 'filename'].tolist()
r_320x241 = df_resolutions.loc[df_resolutions['image_size'] == '320x241', 'filename'].tolist()
r_320x256 = df_resolutions.loc[df_resolutions['image_size'] == '320x256', 'filename'].tolist()


In [ ]:
# observation d'une grille d'images de resolutions 320x240
IMAGE_DIR = full_images
W, H = 224, 224
show_gallery(r_320x240, n=10)

In [ ]:
show_gallery(r_320x213, n=10)

In [ ]:
show_gallery(r_320x214, n=10)

In [ ]:
show_gallery(r_320x180, n=10)

In [ ]:
show_gallery(r_320x212, n=10)

In [ ]:
show_gallery(r_320x239, n=10)

In [ ]:
show_gallery(r_320x320, n=10)

In [ ]:
show_gallery(r_320x211, n=10)

In [ ]:
show_gallery(r_320x241, n=10)


In [ ]:
show_gallery(r_320x256, n=10)

La résolution 320x320 contient en proportion beaucoup plus d'images prises au microscope que les autres résoutions, celle-ci pourrait être écartées ainsi que les résolutions contenant peu d'images (ne faisant pas partie des 5 premières). 

In [ ]:
# création d'une liste d'images dont la résolution n'est pas dans les 10 résolutions les plus répandues ni 320x320
liste_resolutions = ['320x240', '320x213', '320x214', '320x180', '320x212']
df_resolutions_filtered = df_resolutions.loc[~df_resolutions['image_size'].isin(liste_resolutions)]
resolutions_to_delete = df_resolutions_filtered['filename'].tolist()

# mise à jour de la liste d'images à supprimer
for filename in resolutions_to_delete:
    images_to_delete.append(filename)

## Recherche d'identification de la cible (comestible) en fonction des canaux de couleurs

### Analyse des canaux de couleurs

In [ ]:
# création d'une copie reduite du dataset pour l'étude des canaux de couleurs
os.mkdir(full_images + '\channels_study')
dossier_destination = full_images + '\channels_study'

test_channels = os.listdir(full_images)
test_channels = random.sample(test_channels, int(len(test_channels) / 50))

In [ ]:
# copie des images vers dossier_destination
for image_name in test_channels:
    original_image_path = os.path.join(full_images, image_name)
    new_image_path = os.path.join(dossier_destination, image_name)
    shutil.copy(original_image_path, new_image_path)

# resize des images pou robtenir des dimensions homogènes
W, H = 224, 224
for file in os.listdir(dossier_destination):
        image = cv2.imread(os.path.join(dossier_destination, file))
        image_resize = cv2.resize(image, (W, H))
        cv2.imwrite(os.path.join(dossier_destination, file), image_resize)  

In [ ]:
# création du dataframe
filename = os.listdir(dossier_destination)
target = pd.read_csv(dataframe_dataset)
target = target.drop(['kingdom', 'phylum', 'classes', 'order', 'family', 'genus', 'species'], axis = 1)
target.rename(columns = {'image_lien' : 'filename'}, inplace =True)
df = target.loc[target["filename"].isin(filename)]
df.reset_index(drop = True, inplace = True)
df.head()



In [ ]:
# Créer une liste vide pour stocker les chemins d'accès complets
paths = []
# Itérer sur les fichiers du dossier
for file in os.listdir(dossier_destination):
    # Obtenir le chemin d'accès complet du fichier
    path = os.path.join(dossier_destination, file)
    # Ajouter le chemin d'accès complet à la liste
    paths.append(path)
    
image_paths = paths
data = np.empty((len(paths), W, H, 3))

# Itérer sur les images
for i, image_path in enumerate(image_paths):
    image = plt.imread(image_path)
    image_array = image.reshape((W, H, 3))
    data[i] = image_array
    
# Créer trois array NumPy vides
red = np.empty((len(paths), W, H))
green = np.empty((len(paths), W, H))
blue = np.empty((len(paths), W, H))

# Itérer sur les images et extraire les canaux de couleur
for i in range(data.shape[0]):
    red[i] = data[i, :, :, 0]
    green[i] = data[i, :, :, 1]
    blue[i] = data[i, :, :, 2]
    
# Créer un dictionnaire vide
occurrences_red = {}
occurrences_green = {}
occurrences_blue = {}
# Itérer sur les valeurs de l'array NumPy
for value in red.flatten():
    if value not in occurrences_red:
        occurrences_red[value] = 0
    occurrences_red[value] += 1
for value in green.flatten():
    if value not in occurrences_green:
        occurrences_green[value] = 0
    occurrences_green[value] += 1    
for value in blue.flatten():
    if value not in occurrences_blue:
        occurrences_blue[value] = 0
    occurrences_blue[value] += 1

red_hist = {}
for key in sorted(occurrences_red.keys()):
    red_hist[key] = occurrences_red[key]
red_list = list(red_hist.values())
green_hist = {}
for key in sorted(occurrences_green.keys()):
    green_hist[key] = occurrences_green[key]
green_list = list(green_hist.values())
blue_hist = {}
for key in sorted(occurrences_blue.keys()):
    blue_hist[key] = occurrences_blue[key]
blue_list = list(blue_hist.values())

d = {'Red' : pd.Series(red_list), 'Green':pd.Series(green_list), 'Blue':pd.Series(blue_list)}
df_RGBhist = pd.DataFrame(data = d)

# création d'une figure permettant d'observer la répartition des valeurs des canaux
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.bar(df_RGBhist.index, df_RGBhist['Red'], color = 'red', alpha = 0.4)
ax.bar(df_RGBhist.index, df_RGBhist['Green'], color = 'green', alpha = 0.4)
ax.bar(df_RGBhist.index, df_RGBhist['Blue'], color = 'blue', alpha = 0.6)
ax.set_xlabel('value')
ax.yaxis.set_visible(False)
ax.set_title('Répartition des valeurs des canaux de la base de données')
plt.legend(['red', 'green', 'blue'], loc='upper right')
plt.show()

Les 3 canaux de couleurs sont assez similaires, il ne semble pas opportun de supprimer l'un ou l'autre de ces canaux de la base de données d'images.

### Analyse des canaux de couleurs par catégorie pour la partie comestible

In [ ]:
filename = os.listdir(dossier_destination)
# définir la cible pour charger le dataframe correspondant au dossier d'image complet avec les labels de l'étude
target = pd.read_csv(dataframe_dataset)
target = target.drop(['kingdom', 'phylum', 'classes', 'family', 'genus', 'species'], axis = 1)
target.rename(columns = {'image_lien' : 'filename'}, inplace =True)
df = target.loc[target["filename"].isin(filename)]
df.reset_index(drop = True, inplace = True)
#df.head()

# création de sous_dossier catégoriels en fonction de la cible 'comestible'
path_edible = os.path.join(dossier_destination, "edible")
path_inedible = os.path.join(dossier_destination, "inedible")
os.mkdir(path_edible)
os.mkdir(path_inedible)

# extraction des noms d'images du dataframe selon la colonne cible df['edible']
# Sélection les lignes pour edible = 1 et edible = 0
df_edible = df.loc[df["edible"] == 1]
df_inedible = df.loc[df["edible"] == 0]

# Extraction des noms des images dans des listes
images_names_edible = df_edible["filename"].values
images_names_inedible = df_inedible["filename"].values
images_names_edible = list(images_names_edible)
images_names_inedible = list(images_names_inedible)

# copie des fichiers images vers les dossiers de destination
for image_name in images_names_edible :    
    # trouver l'image dans le dossier d'origine
    original_image_path = os.path.join(dossier_destination, image_name)
    # copie de l'image dans le dossier de destination
    new_image_path = os.path.join(path_edible, image_name)
    if os.path.exists(original_image_path):
        shutil.copy(original_image_path, new_image_path)
        
for image_name in images_names_inedible :    
    # trouver l'image dans le dossier d'origine
    original_image_path = os.path.join(dossier_destination, image_name)
    # copie de l'image dans le dossier de destination
    new_image_path = os.path.join(path_inedible, image_name)
    if os.path.exists(original_image_path):
        shutil.copy(original_image_path, new_image_path)
        
# Créer un DataFrame contenant les valeurs des canaux de couleurs des images edible
images_edible = os.listdir(path_edible)
data_edible = []
for image in images_edible:
    image = plt.imread(os.path.join(path_edible, image))
    r, g, b = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    data_edible.append([np.mean(r), np.mean(g), np.mean(b)])
df_edible = pd.DataFrame(data_edible, columns=["red", "green", "blue"])


# Créer un DataFrame contenant les valeurs des canaux de couleurs des images inedible
images_inedible = os.listdir(path_inedible)
data_inedible = []
for image in images_inedible:
    image = plt.imread(os.path.join(path_inedible, image))
    r, g, b = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    data_inedible.append([np.mean(r), np.mean(g), np.mean(b)])
df_inedible = pd.DataFrame(data_inedible, columns=["red", "green", "blue"])

# Création d'un violinplot représentant les canaux de couleur par label cible
fig, ax = plt.subplots(1, 1)

# génération des données du violinplot
data1 = df_edible['red'].tolist()
data2 = df_edible['green'].tolist()
data3 = df_edible['blue'].tolist()
data4 = [300, 300, 300]
data5 = df_inedible['red'].tolist()
data6 = df_inedible['green'].tolist()
data7 = df_inedible['blue'].tolist()

# liste des données à ploter
data = [data1, data2, data3, data4, data5, data6, data7]

# définition de la couleur des violons
colors = ['#FF0000', 'Green', 'Blue','White', '#FF0000', 'Green', 'Blue']
ticks_colors = ['black','black','black','White','black','black','black',]
# Creation du violinplot
plots = ax.violinplot(data, vert=True, showmedians=True, showextrema=False, widths=1, showmeans=True)

# Définition de paramtres personnalisés pour les violons
for pc, color in zip(plots['bodies'], colors):
    pc.set_facecolor(color)
    pc.set_alpha(1)
    pc.set_edgecolor('black')

plots['cmedians'].set_colors(ticks_colors)
plots['cmeans'].set_colors(ticks_colors)

# Definition des paramètres de la figure
ax.set_xticks([1, 2, 3, 4, 5, 6, 7], labels=['', 'Edible', '', '', '', 'Inedible', ''])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_visible(True)
ax.spines['bottom'].set_visible(False)
ax.set_ylim(0, 256)

# Ajout de la légende
red_patch = mpatches.Patch(color='red', label='red')
green_patch = mpatches.Patch(color='green', label='green')
blue_patch = mpatches.Patch(color='blue', label='blue')
ax.legend(handles=[red_patch, green_patch, blue_patch], loc = 'upper center', title = 'Channels', ncol = 3)
plt.show()

Les canaux RGB des cibles de l'étude sont assez similaires, il ne semble pas y avoir de corrélations entre les canaux de couleurs et la comestibilité des champignons.

## Recherche d'identification de la cible (order) en fonction des canaux de couleurs

### Analyse des canaux de couleurs par catégorie pour la partie 'order'

In [ ]:
filename = os.listdir(full_images)
# définir la cible pour charger le dataframe correspondant au dossier d'image complet avec les labels de l'étude
target = pd.read_csv(r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\dataset.csv')
target = target.drop(['kingdom', 'phylum', 'classes', 'family', 'genus', 'species'], axis = 1)
target = target.drop(target[target['edible'] == 0].index)
target.rename(columns = {'image_lien' : 'filename'}, inplace =True)
df = target.loc[target["filename"].isin(filename)]
df.reset_index(drop = True, inplace = True)
#df.head()

# Création d'une liste des catégories de 'order'
order_list = df.order.unique()

# Création du dossier et de la liste d'images à copier (max 2000 éléments tirés aléatoirement)
for order in order_list :
    path = os.path.join(full_images, order)
    os.mkdir(path)
    images_names = list(df.loc[df["order"] == order, 'filename'])
    if len(images_names) > 1500 :
        images_names = random.sample(images_names, 1500)

    
    # copie des fichiers images vers les dossiers de destination
    for image_name in images_names :    
        # trouver l'image dans le dossier d'origine
        original_image_path = os.path.join(full_images, image_name)
        # copie de l'image dans le dossier de destination
        new_image_path = os.path.join(path, image_name)
        if os.path.exists(original_image_path):
            image = PIL.Image.open(original_image_path)
            if image.mode == 'RGB':
                shutil.copy(original_image_path, new_image_path)

        
# Créer des DataFrames contenant les valeurs des canaux de couleurs des images de chaque catégorie
order_list = df.order.unique()
path = os.path.join(full_images, order_list[0])
images = os.listdir(path)
data = []
for image in images:
    image = plt.imread(os.path.join(path, image))
    data.append([np.mean(image[:, :, 0]), np.mean(image[:, :, 1]), np.mean(image[:, :, 2])])
Agaricales = pd.DataFrame(data, columns=["red", "green", "blue"])

path = os.path.join(full_images, order_list[1])
images = os.listdir(path)
data = []
for image in images:
    image = plt.imread(os.path.join(path, image))
    data.append([np.mean(image[:, :, 0]), np.mean(image[:, :, 1]), np.mean(image[:, :, 2])])
Polyporales = pd.DataFrame(data, columns=["red", "green", "blue"])

path = os.path.join(full_images, order_list[2])
images = os.listdir(path)
data = []
for image in images:
    image = plt.imread(os.path.join(path, image))
    data.append([np.mean(image[:, :, 0]), np.mean(image[:, :, 1]), np.mean(image[:, :, 2])])
Russulales = pd.DataFrame(data, columns=["red", "green", "blue"])

path = os.path.join(full_images, order_list[3])
images = os.listdir(path)
data = []
for image in images:
    image = plt.imread(os.path.join(path, image))
    data.append([np.mean(image[:, :, 0]), np.mean(image[:, :, 1]), np.mean(image[:, :, 2])])
Gomphales = pd.DataFrame(data, columns=["red", "green", "blue"])

path = os.path.join(full_images, order_list[4])
images = os.listdir(path)
data = []
for image in images:
    image = plt.imread(os.path.join(path, image))
    data.append([np.mean(image[:, :, 0]), np.mean(image[:, :, 1]), np.mean(image[:, :, 2])])
Pezizales = pd.DataFrame(data, columns=["red", "green", "blue"])

path = os.path.join(full_images, order_list[5])
images = os.listdir(path)
data = []
for image in images:
    image = plt.imread(os.path.join(path, image))
    data.append([np.mean(image[:, :, 0]), np.mean(image[:, :, 1]), np.mean(image[:, :, 2])])
Boletales = pd.DataFrame(data, columns=["red", "green", "blue"])

path = os.path.join(full_images, order_list[6])
images = os.listdir(path)
data = []
for image in images:
    image = plt.imread(os.path.join(path, image))
    data.append([np.mean(image[:, :, 0]), np.mean(image[:, :, 1]), np.mean(image[:, :, 2])])
Cantharellales = pd.DataFrame(data, columns=["red", "green", "blue"])

path = os.path.join(full_images, order_list[7])
images = os.listdir(path)
data = []
for image in images:
    image = plt.imread(os.path.join(path, image))
    data.append([np.mean(image[:, :, 0]), np.mean(image[:, :, 1]), np.mean(image[:, :, 2])])
Auriculariales = pd.DataFrame(data, columns=["red", "green", "blue"])

# CRéation de la figure et des données du violinplot
fig, ax = plt.subplots(1, 1, figsize = (9, 4))

data1 = Agaricales['red'].tolist()
data2 = Agaricales['green'].tolist()
data3 = Agaricales['blue'].tolist()
data4 = [300, 300, 300]
data5 = Polyporales['red'].tolist()
data6 = Polyporales['green'].tolist()
data7 = Polyporales['blue'].tolist()
data8 = [300, 300, 300]
data9 = Russulales['red'].tolist()
data10 = Russulales['green'].tolist()
data11 = Russulales['blue'].tolist()
data12 = [300, 300, 300]
data13 = Gomphales['red'].tolist()
data14 = Gomphales['green'].tolist()
data15 = Gomphales['blue'].tolist()
data16 = [300, 300, 300]
data17 = Pezizales['red'].tolist()
data18 = Pezizales['green'].tolist()
data19 = Pezizales['blue'].tolist()
data20 = [300, 300, 300]
data21 = Boletales['red'].tolist()
data22 = Boletales['green'].tolist()
data23 = Boletales['blue'].tolist()
data24 = [300, 300, 300]
data25 = Cantharellales['red'].tolist()
data26 = Cantharellales['green'].tolist()
data27 = Cantharellales['blue'].tolist()
data28 = [300, 300, 300]
data29 = Auriculariales['red'].tolist()
data30 = Auriculariales['green'].tolist()
data31 = Auriculariales['blue'].tolist()

# Creation de la liste des données a représenter
data = [data1, data2, data3, data4, data5, data6, data7, data8, data9, data10,
        data11, data12, data13, data14, data15, data16, data17, data18, data19, data20,
        data21, data22, data23, data24, data25, data26, data27, data28, data29, data30,
        data31]

# Définition des couleurs des violons
colors = ['Red', 'Green', 'Blue','White',
          'Red', 'Green', 'Blue', 'White',
         'Red', 'Green', 'Blue','White',
         'Red', 'Green', 'Blue','White',
         'Red', 'Green', 'Blue','White',
         'Red', 'Green', 'Blue','White',
         'Red', 'Green', 'Blue','White',
         'Red', 'Green', 'Blue']
ticks_colors = ['black','black','black','White',
                'black','black','black','White',
                'black','black','black','White',
                'black','black','black','White',
                'black','black','black','White',
                'black','black','black','White',
                'black','black','black','White',
                'black','black','black',]

# Creation du violinplot
plots = ax.violinplot(data, vert=True, showmedians=True, showextrema=False, widths=1, showmeans=True)

# Définition personnalisée des couleurs des violons
for pc, color in zip(plots['bodies'], colors):
    pc.set_facecolor(color)
    pc.set_alpha(1)
    pc.set_edgecolor('black')

plots['cmedians'].set_colors(ticks_colors)
plots['cmeans'].set_colors(ticks_colors)

# Définition des labels de l'axe des x
ax.set_xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
              labels=['', 'Agaricales', '', '',
                      '', 'Polyporales', '', '',
                     '', 'Russulales', '', '',
                     '', 'Gomphales', '', '',
                     '', 'Pezizales', '', '',
                     '', 'Boletales', '', '',
                     '', 'Cantharellales', '', '',
                     '', 'Auriculariales', ''])

# définition de quelques paramètres de la figure
ax.xaxis.set_tick_params(labelsize = 8) 
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_visible(True)
ax.spines['bottom'].set_visible(False)
ax.set_ylim(0, 256)

# ajout de la légende
red_patch = mpatches.Patch(color='red', label='red')
green_patch = mpatches.Patch(color='green', label='green')
blue_patch = mpatches.Patch(color='blue', label='blue')
ax.legend(handles=[red_patch, green_patch, blue_patch], loc = 'upper center', title = 'Channels', ncol = 3)
plt.show()

Les canaux RGB des cibles de l'étude sont assez similaires, il ne semble pas y avoir de corrélations entre les canaux de couleurs et la comestibilité des champignons.

# Nettoyage du dataset

### Pour le dataset d'images 

In [ ]:
# affichage du nonmbre d'élements à supprimer
print(len(images_to_delete), "images peuvent être supprimées du dataset d'images")

In [ ]:
# suppression des images
files = os.listdir(full_images)
for file in files:
    if file in images_to_delete:
        os.remove(os.path.join(full_images, file))

### Pour le dataframe 

In [ ]:
files = os.listdir(full_images)
# mise à jour du dataframe
df = pd.read_csv(dataframe_dataset)
df_cleaned = df.loc[df['filename'].isin(files)]
dossier_parent = os.path.dirname(full_images)
df_cleaned.to_csv(f"{dossier_parent}\cleaned_cleaned_dataset.csv", index = False)

#### Controle de la concordance des données et suppression des fichiers et dossiers créés pour cette étude 

In [ ]:
# suppression des sous-dossiers de full_images créés pour cette étude

# Parcourir les dossiers du dossier
for child in os.listdir(full_images):
    # Obtenir le chemin du dossier enfant
    chemin_dossier_enfant = os.path.join(full_images, child)

    # Supprimer le dossier enfant et son contenu
    try:
        shutil.rmtree(chemin_dossier_enfant)
    except OSError as e:
        pass        

In [ ]:
# controle de la correspondance entre le datafarme créé et le dossier d'images
count = 0
for file in os.listdir(full_images):
    # Vérifier si le fichier est un sous-dossier
    if not os.path.isdir(os.path.join(full_images, file)):
        # Incrémenter le compteur
        count += 1
print("Le dataframe de l'étude contient",len(df_cleaned),"lignes")
print("Le dataset d'images contient",count,"images")